# Aplicando conceitos do artigo  do *Rizzi*



 Alunos: Adriel Ferreira Trajano e Kamily Assis


## Bibliotecas

In [12]:
import os
from pydantic import BaseModel
from pydantic.fields import Field
from openai import OpenAI
from dotenv import load_dotenv
import prompts
import context_prompts
import re
import yaml

## Variáveis de Ambiente 

In [13]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = OPENAI_API_KEY)

## Chamada de API da OpenAI

In [4]:
def answer(msg):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a data warehouse designer. I’m the end-user."
            },
            {
                "role": "user", 
                "content": msg
            }
        ]
    )
    return response.choices[0].message.content

In [5]:
def answer_RQ1(msg):
    response = client.chat.completions.create(
        
        model = "gpt-4o-mini",
        temperature = 0,
        messages=[
            {
                "role": "user",
                "content": msg
            }
        ]
    )
    return response.choices[0].message.content

## Funções auxiliares

In [6]:
def save_to_file(filename, content):
    """
    Saves content to a file.
    """
    with open(filename, 'w') as file:
        file.write(content)

In [7]:
filename = "input/file.yml"
with open(filename, 'r') as file:
    content = file.read()

In [ ]:
def extract_yaml_content(response_text):
    pattern = r'```yaml(.*?)```'
    match = re.search(pattern, response_text, re.DOTALL)
    
    if match:
        return match.group(1).strip()  
    else:
        pattern_fallback = r'```(.*?)```'
        match_fallback = re.search(pattern_fallback, response_text, re.DOTALL)
        return match_fallback.group(1).strip() if match_fallback else response_text

## Automação da Metodologia

In [ ]:
for q_prompt in dir(prompts):

    if q_prompt.startswith("RQ1"):
        prompt_value = getattr(prompts, q_prompt)
        answer_text = answer_RQ1(prompt_value)
        save_to_file(f"output/{q_prompt}.txt", answer_text)


    for context_prompt in dir(context_prompts):
        
        if context_prompt.startswith("__") or q_prompt.startswith("__"):  
            continue

        if context_prompt.endswith("q2") and q_prompt.startswith("RQ2"):
            combined_prompt = getattr(context_prompts, context_prompt).format(
                content=content,
                msg=getattr(prompts, q_prompt)
            )
            answer_text = answer(combined_prompt)
            answer_text = extract_yaml_content(answer_text)
            var_name = f"{context_prompt}_{q_prompt}"
            save_to_file(f"output/{var_name}.yaml", answer_text)

            filename = f"output/{var_name}.yaml"
            with open(filename, 'r') as file:
                content = file.read()

        elif context_prompt.endswith("q3") and q_prompt.startswith("RQ3"):
            combined_prompt = getattr(context_prompts, context_prompt).format(
                content=content,
                msg=getattr(prompts, q_prompt)
            )
            answer_text = answer(combined_prompt)
            answer_text = extract_yaml_content(answer_text)
            var_name = f"{context_prompt}_{q_prompt}"
            save_to_file(f"output/{var_name}.yaml", answer_text)

            filename = f"output/{var_name}.yaml"
            with open(filename, 'r') as file:
                content = file.read()
    
    